In [8]:
import time
import requests
import pymysql


HOST = "localhost"
USER = "root"
PASSWORD = ""
DB_NAME = "harvard_artifacts"

# ---------- API ----------
API_KEY = "20f6731f-61df-45d8-9a01-b48944b6ec56"
BASE_URL = "https://api.harvardartmuseums.org"

# ---------- SQL ----------
SQL_META = """
INSERT IGNORE INTO artifact_metadata
(id, title, culture, period, century, medium, dimensions, description,
 department, classification, accessionyear, accessionmethod)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

SQL_MEDIA = """
INSERT IGNORE INTO artifact_media
(objectid, imagecount, mediacount, colorcount, rank, datebegin, dateend)
VALUES (%s,%s,%s,%s,%s,%s,%s)
"""

SQL_COLOR = """
INSERT IGNORE INTO artifact_colors
(objectid, color, spectrum, hue, percent, css3)
VALUES (%s,%s,%s,%s,%s,%s)
"""

# ---------- DB Helper ----------
def get_conn():
    return pymysql.connect(host=HOST, user=USER, password=PASSWORD, database=DB_NAME)

# ---------- Extract ----------
def fetch_artifacts(classification="Paintings", size=5, page=1):
    """Fetch artifacts from Harvard API (one page)"""
    url = f"{BASE_URL}/object"
    params = {
        "apikey": API_KEY,
        "classification": classification,
        "size": size,
        "page": page
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json().get("records", [])

# ---------- Transform ----------
def transform_records(records):
    """Transform JSON into tuples for SQL insertion"""
    metadata_list, media_list, color_list = [], [], []

    for obj in records:
        oid = obj.get("objectid")
        if not oid:
            continue

        metadata_list.append((
            oid, obj.get("title"), obj.get("culture"), obj.get("period"),
            obj.get("century"), obj.get("medium"), obj.get("dimensions"),
            obj.get("description"), obj.get("department"),
            obj.get("classification"), obj.get("accessionyear"),
            obj.get("accessionmethod")
        ))

        media_list.append((
            oid, obj.get("imagecount"), obj.get("mediacount"),
            obj.get("colorcount"), obj.get("rank"),
            obj.get("datebegin"), obj.get("dateend")
        ))

        for c in obj.get("colors", []) or []:
            color_list.append((
                oid, c.get("color"), c.get("spectrum"),
                c.get("hue"), c.get("percent"), c.get("css3")
            ))

    return metadata_list, media_list, color_list

# ---------- Load ----------
def insert_into_db(metadata, media, colors):
    conn = get_conn()
    cur = conn.cursor()

    if metadata:
        cur.executemany(SQL_META, metadata)
    if media:
        cur.executemany(SQL_MEDIA, media)
    if colors:
        cur.executemany(SQL_COLOR, colors)

    conn.commit()
    cur.close()
    conn.close()
    print(f"Inserted {len(metadata)} metadata, {len(media)} media, {len(colors)} colors")

# ---------- ETL Pipeline (with pagination) ----------
def run_etl_for_classification(classification="Paintings", target=2500, page_size=100, pause=0.2):
    total_inserted = 0
    page = 1
    conn = get_conn()
    cur = conn.cursor()

    try:
        while total_inserted < target:
            url = f"{BASE_URL}/object"
            params = {
                "apikey": API_KEY,
                "classification": classification,
                "size": page_size,
                "page": page
            }
            r = requests.get(url, params=params, timeout=30)
            r.raise_for_status()
            data = r.json()

            records = data.get("records", [])
            if not records:
                break

            meta_batch, media_batch, color_batch = transform_records(records)
            if meta_batch:
                cur.executemany(SQL_META, meta_batch)
            if media_batch:
                cur.executemany(SQL_MEDIA, media_batch)
            if color_batch:
                cur.executemany(SQL_COLOR, color_batch)

            conn.commit()
            total_inserted += len(meta_batch)
            print(f"Page {page}: inserted {len(meta_batch)} rows (total={total_inserted})")

            info = data.get("info", {})
            if page >= info.get("pages", 0):
                break

            page += 1
            time.sleep(pause)

    finally:
        cur.close()
        conn.close()

    print(f"ETL completed for '{classification}', total={total_inserted}")

# ---------- Script Runner ----------
if __name__ == "__main__":
    # Test small batch
     #raw = fetch_artifacts("Paintings", size=5, page=1)
     #meta, media, colors = transform_records(raw)
     #insert_into_db(meta, media, colors)

    # run full ETL
    run_etl_for_classification("Paintings", target=2500)

Page 1: inserted 100 rows (total=100)
Page 2: inserted 100 rows (total=200)
Page 3: inserted 100 rows (total=300)
Page 4: inserted 100 rows (total=400)
Page 5: inserted 100 rows (total=500)
Page 6: inserted 100 rows (total=600)
Page 7: inserted 100 rows (total=700)
Page 8: inserted 100 rows (total=800)
Page 9: inserted 100 rows (total=900)
Page 10: inserted 100 rows (total=1000)
Page 11: inserted 100 rows (total=1100)
Page 12: inserted 100 rows (total=1200)
Page 13: inserted 100 rows (total=1300)
Page 14: inserted 100 rows (total=1400)
Page 15: inserted 100 rows (total=1500)
Page 16: inserted 100 rows (total=1600)
Page 17: inserted 100 rows (total=1700)
Page 18: inserted 100 rows (total=1800)
Page 19: inserted 100 rows (total=1900)
Page 20: inserted 100 rows (total=2000)
Page 21: inserted 100 rows (total=2100)
Page 22: inserted 100 rows (total=2200)
Page 23: inserted 100 rows (total=2300)
Page 24: inserted 100 rows (total=2400)
Page 25: inserted 100 rows (total=2500)
ETL completed for 